# 1. Com a aplicação criada em Python do exercício 6 da folha de exercícios 3, use a biblioteca Pandas para:
### - Guardar os conteúdos numa base de dados das pessoas inseridas
### - Ler o conteúdo da base de dados
### - Exportar para ficheiro csv as pessoas inseridas
### - Filtrar os elementos que aparecem na tabela com os seguintes critérios, usando uma interface com o utilizador:
### - Correspondência com a profissão
### - Com a idade. Esta pode ser maior, menor ou igual

In [1]:
import os
import csv
import sqlite3

class Pessoa:
    def __init__(self, primeiroNome, ultimoNome, idade, profissao):
        self.primeiroNome = primeiroNome
        self.ultimoNome = ultimoNome
        self.idade = idade
        self.profissao = profissao

# Conectar ao banco de dados SQLite
db_connection = sqlite3.connect('pessoas.db')
cursor = db_connection.cursor()

# Criar a tabela para armazenar as informações das pessoas
cursor.execute('''
    CREATE TABLE IF NOT EXISTS pessoas (
        primeiroNome TEXT,
        ultimoNome TEXT,
        idade INTEGER,
        profissao TEXT
    )
''')
db_connection.commit()

def adicionar_pessoa(pessoa):
    cursor.execute('INSERT INTO pessoas VALUES (?, ?, ?, ?)', (pessoa.primeiroNome, pessoa.ultimoNome, pessoa.idade, pessoa.profissao))
    db_connection.commit()

def listar_pessoas():
    cursor.execute('SELECT * FROM pessoas')
    return cursor.fetchall()

def exportar_para_csv(pessoas, nome_arquivo):
    with open(nome_arquivo, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Primeiro Nome', 'Último Nome', 'Idade', 'Profissão'])
        for pessoa in pessoas:
            csv_writer.writerow([pessoa[0], pessoa[1], pessoa[2], pessoa[3]])

def filtrar_pessoas_por_profissao(pessoas, profissao):
    return [pessoa for pessoa in pessoas if pessoa[3] == profissao]

def filtrar_pessoas_por_idade(pessoas, operador, idade):
    if operador == '<':
        return [pessoa for pessoa in pessoas if pessoa[2] < idade]
    elif operador == '>':
        return [pessoa for pessoa in pessoas if pessoa[2] > idade]
    elif operador == '=':
        return [pessoa for pessoa in pessoas if pessoa[2] == idade]

while True:
    print('****************************************')
    print('*************User interface*************')
    print('****************************************')
    print('C - Consultar pessoas')
    print('T - Adicionar pessoas')
    print('E - Exportar para CSV')
    print('F - Filtrar pessoas')
    print('X - Sair')
    opcao = input('Escolha uma opção: ')

    if opcao == 'C':
        pessoas = listar_pessoas()
        if not pessoas:
            print('Não existem pessoas adicionadas')
        else:
            print('----------Consulta de pessoas--------------')
            for pessoa in pessoas:
                print('Primeiro Nome:', pessoa[0])
                print('Último Nome:', pessoa[1])
                print('Idade:', pessoa[2])
                print('Profissão:', pessoa[3])
                print('-------------------------------------------')
        input('Pressione Enter para continuar...')

    elif opcao == 'T':
        print('-----------Introduzir pessoas--------------')
        pNome = input('Introduza o primeiro nome: ')
        uNome = input('Introduza o último nome: ')
        idade = int(input('Introduza a idade: '))
        profissao = input('Introduza a profissão: ')
        nova_pessoa = Pessoa(pNome, uNome, idade, profissao)
        adicionar_pessoa(nova_pessoa)
        print('Pessoa nova introduzida.')
        input('Pressione Enter para continuar...')

    elif opcao == 'E':
        pessoas = listar_pessoas()
        if not pessoas:
            print('Não existem pessoas para exportar')
        else:
            nome_arquivo = input('Nome do arquivo CSV para exportar: ')
            exportar_para_csv(pessoas, nome_arquivo)
            print('Pessoas exportadas para', nome_arquivo)
            input('Pressione Enter para continuar...')

    elif opcao == 'F':
        sub_opcao = input('Filtrar por (P)rofissão ou (I)dade? ').upper()
        if sub_opcao == 'P':
            profissao = input('Profissão para filtrar: ')
            pessoas_filtradas = filtrar_pessoas_por_profissao(listar_pessoas(), profissao)
            if not pessoas_filtradas:
                print('Nenhuma correspondência encontrada para a profissão', profissao)
            else:
                print('Pessoas com profissão', profissao, ':')
                for pessoa in pessoas_filtradas:
                    print(pessoa[0], pessoa[1], pessoa[2])
        elif sub_opcao == 'I':
            operador = input('Operador de idade (<, >, =): ')
            idade = int(input('Idade para filtrar: '))
            pessoas_filtradas = filtrar_pessoas_por_idade(listar_pessoas(), operador, idade)
            if not pessoas_filtradas:
                print('Nenhuma correspondência encontrada para a idade', operador, idade)
            else:
                print('Pessoas com idade', operador, idade, ':')
                for pessoa in pessoas_filtradas:
                    print(pessoa[0], pessoa[1], pessoa[2])
        else:
            print('Opção inválida')
        input('Pressione Enter para continuar...')

    elif opcao == 'X':
        db_connection.close()
        break


****************************************
*************User interface*************
****************************************
C - Consultar pessoas
T - Adicionar pessoas
E - Exportar para CSV
F - Filtrar pessoas
X - Sair
